You can download market data for various cryptocurrency pairs from Binance with your API keys via a beautiful tkinter GUI.

I use this notebook to gather historical price data to backtest my algorithmic trading strategies.

---

In [1]:
import csv
from binance import Client

from tkinter import *
import tkinter as tk
from tkcalendar import *
import datetime
from datetime import date
from datetime import datetime as dt

---

Tkinter GUI for selecting a timeframe for history market data.

In [2]:
today = date.today()
mindate = date(year = 2020,
                        month = 1,
                        day = 1)
maxdate = today - datetime.timedelta(days = 1)

selectedStartDate = date.today()
selectedEndDate = date.today()

root = Tk()
root.title('Date Picker')


cal = Calendar(root, selectmode="day",
               year=dt.today().year, month=dt.now().month,
               day=dt.now().day, mindate = mindate, maxdate = maxdate)
cal.pack(pady=20)

def set_start_date():
    global selectedStartDate
    start_date.config(text=cal.get_date())
    selectedStartDate = cal.get_date()
def set_end_date():
    global selectedEndDate
    end_date.config(text=cal.get_date())
    selectedEndDate = cal.get_date()
def quit():
    root.destroy()

startdate_btn = Button(root, text='Set Start Date', command=set_start_date)
enddate_btn = Button(root,text='Set End Date', command=set_end_date)
startdate_btn.pack(side=tk.LEFT)
enddate_btn.pack(side=tk.RIGHT)
quit_btn = Button(root,text='Quit', command=quit)
quit_btn.pack(side=tk.BOTTOM)

start_date = Label(root,text="")
start_date.pack(pady=20, padx=0)
end_date = Label(root,text="")
end_date.pack(pady=20, padx= 40)

root.mainloop()

---

Cleaning and processing date data for binance.client

In [3]:
months_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
splitStartDate = selectedStartDate.split('/')
splitEndDate = selectedEndDate.split('/')
startDay = splitStartDate[1]+' '
endDay = splitEndDate[1]+' '
startMonth = months_list[int(splitStartDate[0])-1]+', '
endMonth = months_list[int(splitEndDate[0])-1]+', '
startYear = '20'+splitStartDate[2]
endYear = '20'+splitEndDate[2]

DATE_START = (startDay+startMonth+startYear)
DATE_END = (endDay+endMonth+endYear)

---

Selecting a currency pair that we want to retrieve market data from.

In [4]:
currency_pairs = ['BTCUSDT','ETHUSDT','BNBUSDT','ADAUSDT','SOLUSDT','XRPUSDT','DOTUSDT']

CURRENCY_PAIR = 'BTCUSDT' # Default currency pair is BTC/USDT

root = Tk()

dropdown_element = StringVar(root)
dropdown_element.set(currency_pairs[0]) # Set default element (BTCUSDT)

dropdown = OptionMenu(root, dropdown_element, *currency_pairs)
dropdown.pack()

def save_selected():
    global CURRENCY_PAIR
    CURRENCY_PAIR = dropdown_element.get()
    root.destroy()
    
select_btn = Button(root, text="Select", command=save_selected)
select_btn.pack(pady = 20)

mainloop()

# Insert your Binance API Key and Secret to connect your account.

In [6]:
API_KEY = '***'
API_SECRET = '***'
client = Client(API_KEY, API_SECRET)

datafile = 'data/'+CURRENCY_PAIR+'_'+INTERVAL+' '+DATE_START+' - '+DATE_END+'.csv'
csvfile = open(datafile, 'w', newline='')
candle_writer = csv.writer(csvfile, delimiter=',')

candles = client.get_historical_klines(CURRENCY_PAIR, intervals[INTERVAL], DATE_START, DATE_END)

for candle in candles:
    candle[0] = candle[0] / 1000
    candle_writer.writerow(candle)

csvfile.close()

print(datafile, len(candles))

data/BTCUSDT_1hour 25 Nov, 2020 - 25 Nov, 2021.csv 8743


In [5]:
intervals = {
'1min' : Client.KLINE_INTERVAL_1MINUTE,
'3min' : Client.KLINE_INTERVAL_3MINUTE,
'5min' : Client.KLINE_INTERVAL_5MINUTE,
'15min' : Client.KLINE_INTERVAL_15MINUTE,
'30min' : Client.KLINE_INTERVAL_30MINUTE,
'1hour' : Client.KLINE_INTERVAL_1HOUR,
'2hour' : Client.KLINE_INTERVAL_2HOUR,
'4hour' : Client.KLINE_INTERVAL_4HOUR,
'6hour' : Client.KLINE_INTERVAL_6HOUR,
'8hour' : Client.KLINE_INTERVAL_8HOUR,
'12hour' : Client.KLINE_INTERVAL_12HOUR,
'1-day' : Client.KLINE_INTERVAL_1DAY,
'3-day' : Client.KLINE_INTERVAL_3DAY,
'1-week' : Client.KLINE_INTERVAL_1WEEK,
'1-month' : Client.KLINE_INTERVAL_1MONTH
}

INTERVAL = intervals['1-day'] # Set default interval (1 day)

root = Tk()

dropdown_element = StringVar(root)
dropdown_element.set(intervals['1-day']) # Set default element (1 day)

dropdown = OptionMenu(root, dropdown_element, *intervals)
dropdown.pack()

def save_selected():
    global INTERVAL
    INTERVAL = dropdown_element.get()
    root.destroy()
    
select_btn = Button(root, text='Select', command=save_selected)
select_btn.pack(pady = 20)

mainloop()

---